### Onboarding Waitlist 

In [109]:
import pandas as pd
import numpy as np 

In [111]:
onboarding_data = pd.read_csv('onboarding_waitlist.csv', low_memory=False)
registration_data = pd.read_excel('network_register.xlsx', sheet_name='Sheet1')

print(len(registration_data))
print(len(onboarding_data))

299
200


### Prepare Data and Merge 

In [112]:
# Strip of leading and tailing white space 
onboarding_data["safe_address"] = onboarding_data["safe_address"].str.strip()
registration_data["What is your HOPR safe address?"] = registration_data["What is your HOPR safe address?"].str.strip()

# Make everything lower case letters
onboarding_data["safe_address"] = [x.lower() for x in onboarding_data["safe_address"]]
registration_data["What is your HOPR safe address?"] = [x.lower() for x in registration_data["What is your HOPR safe address?"]] 

# Check for duplicates
dataDup_onboarding = onboarding_data.duplicated(subset=['safe_address'], keep='last')  
dataDup_registration = registration_data.duplicated(subset=['What is your HOPR safe address?'], keep='last')  

# dataDup.value_counts()
onboarding_data['Duplicate'] = dataDup_onboarding
registration_data['Duplicate'] = dataDup_registration 

# Only keep unique values 
onboarding_data_01 = onboarding_data.loc[onboarding_data['Duplicate'] == False]
registration_data_01 = registration_data.loc[registration_data['Duplicate'] == False]

print(onboarding_data_01['Duplicate'].value_counts())
print(registration_data_01['Duplicate'].value_counts())

Duplicate
False    200
Name: count, dtype: int64
Duplicate
False    282
Name: count, dtype: int64


In [113]:
waitlist = registration_data_01.merge(onboarding_data_01, how='left', left_on='What is your HOPR safe address?'
                                                    , right_on='safe_address')

waitlist = waitlist[['Time', 'deployment_date', 'What is your Node address', 'safe_address', 'deployment_tx_hash', 'wxHOPR_balance', 'nr_nft']]

# rename comumns 
waitlist = waitlist.rename(columns={"Time": "registration_time", "What is your Node address": "node_address"})
print(len(waitlist))

# exlude non elidgible nodes 
waitlist_01 = waitlist[~waitlist['safe_address'].isnull()] 
print(len(waitlist_01))
display(waitlist_01.head())

282
93


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
14,2023-09-09T12:56:14.783Z,2023-09-09 12:40:10.000 UTC,0x8de95cdc3291152381f24226422ed9c08beec146,0x4d219619fa660c02236c89e5de46843bdeb41233,0xba138c0798723b959d9491609ab606d1e503d5a438eb...,89000.000000,False
23,2023-09-09T13:27:09.282Z,2023-09-09 12:23:55.000 UTC,0x7ca010ee624b8186e98f41ec2fa6fa329700c104,0xc9f55edb61a23567302e8553f1e5e37574c71e30,0xe76040b340d5afbced1fab05309c64ac710601a3c1a6...,31809.579505,False
38,2023-09-09T14:13:15.202Z,2023-09-04 14:55:25.000 UTC,0x26323877aab52f90c5da19b53320c99627ab2bfc,0x8629dbfd74e0b6016215c064093e3c8259a3b260,0x64750473ebf6c7d3b9d53483f4e89a32a43e3ab8a166...,11425.346894,True
43,2023-09-09T14:31:55.044Z,2023-09-09 14:22:30.000 UTC,0x775175769897c0a46781b423c599f1b6a2b4cde8,0x244454ab29bafb5bbb95f8f6de28fcc45e81b620,0x506226a43ddfffc0bfb48124e7b782232624430010b1...,52186.000000,False
49,2023-09-09T14:54:40.369Z,2023-09-09 14:42:15.000 UTC,0x68cff38fa5d50b13734b1e8d7f26df7ef3bb3f69,0x4ab0895c38380dc61b4553963a84033837cd7b6a,0x8371753b265ab7f3dea55c531a06cdd34bffb5ea5fea...,75000.000000,False


### Delete non-elidgible nodes 

In [114]:
# Not enough stake 
waitlist_02 = waitlist_01[waitlist_01['wxHOPR_balance'] >= 10000]
print(len(waitlist_02))
print(waitlist_02.dtypes)

90
registration_time      object
deployment_date        object
node_address           object
safe_address           object
deployment_tx_hash     object
wxHOPR_balance        float64
nr_nft                 object
dtype: object


In [115]:
waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)
print(waitlist_02['nr_nft'].value_counts())

nr_nft
True     61
False    29
Name: count, dtype: int64


In [116]:
# Users with a stake >= 30000 are eligible regardless of NR NFT
waitlist_03 = waitlist_02.copy() 
waitlist_03.loc[:, 'eligible'] = np.where(waitlist_02.loc[:,'wxHOPR_balance'] >= 30000, True, False)

# Users with a stake < 30000 are eligible if they have a network registry NFT
waitlist_04 = waitlist_03.copy()
waitlist_04.loc[:, 'eligible'] = np.where((waitlist_03.loc[:,'wxHOPR_balance'] < 30000) & (waitlist_03.loc[:,'nr_nft'] == True), True, waitlist_03.loc[:, 'eligible'])

print(waitlist_04.loc[:, 'eligible'].value_counts())


eligible
True     89
False     1
Name: count, dtype: int64


In [117]:
waitlist_05 = waitlist_04[waitlist_04.loc[:,'eligible'] == True]
print(len(waitlist_05))

89


In [118]:
# delete users that registered with their peer_id
valid_address = waitlist_05['node_address'].str.startswith('0x')
waitlist_06 = waitlist_05[valid_address]
print(len(waitlist_06))

82


### Sort Users 

In [119]:
nr_waitlist = waitlist_06[waitlist_06['nr_nft'] == True] 
stake_waitlist =waitlist_06[waitlist_06['nr_nft'] == False]

print(len(nr_waitlist))
print(len(stake_waitlist))

54
28


In [120]:
nr_waitlist_01 = nr_waitlist.sort_values(by="deployment_date", ascending=True).reset_index(drop=True)
print(len(nr_waitlist_01))
display(nr_waitlist_01.head())

54


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-11T14:46:19.272Z,2023-09-04 00:47:25.000 UTC,0xf7adb6cd145c846c3805dc2cd57f1ca6a1800803,0xe540efb03334e43a9bf33c791819c279f8342ecb,0x53f5160b9f9d5e970141f841833862bebdfdb4a916c7...,10000.000000,True,True
1,2023-09-09T14:13:15.202Z,2023-09-04 14:55:25.000 UTC,0x26323877aab52f90c5da19b53320c99627ab2bfc,0x8629dbfd74e0b6016215c064093e3c8259a3b260,0x64750473ebf6c7d3b9d53483f4e89a32a43e3ab8a166...,11425.346894,True,True
2,2023-09-12T20:19:18.691Z,2023-09-04 17:27:20.000 UTC,0xFB2165f4AbEEd496B4EDd3234C4CC09f1557226e,0x3093a8e1621029608e8759f15b74fb18044723ab,0xc8e4ce1c4c9f147a6321d8bd598c355ba5a2095188ca...,26650.897940,True,True
3,2023-09-11T10:29:27.173Z,2023-09-05 17:34:20.000 UTC,0x9bd3c1c34a5599a3e0789453ae1f5f50fbc78c27,0x17b594192539e703febcb218357157d1b6378c7f,0x3ccbc34d480097cd8c2de2ee65498070589f37c6272f...,17069.339900,True,True
4,2023-09-11T13:41:24.101Z,2023-09-06 08:33:40.000 UTC,0xc65525f2Af82eB3D0a5ACBB54b3f7771860f61e0,0x9bf3b5e7f632b60c98836e059558895d3db08556,0x8bcfe5ffee530413ee7dad0f5b53f39d655d6880b02a...,12548.660373,True,True


In [121]:
stake_waitlist_01 = stake_waitlist.sort_values(by="wxHOPR_balance", ascending=False).reset_index(drop=True)
print(len(stake_waitlist))
display(stake_waitlist_01.head())

28


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-09T12:56:14.783Z,2023-09-09 12:40:10.000 UTC,0x8de95cdc3291152381f24226422ed9c08beec146,0x4d219619fa660c02236c89e5de46843bdeb41233,0xba138c0798723b959d9491609ab606d1e503d5a438eb...,89000.000000,False,True
1,2023-09-11T21:47:35.423Z,2023-09-11 20:32:20.000 UTC,0xa9ea78b6b1cce92501a67ec624e19efead9be2e4,0xaf999f3db49054e23c640e6711f3588bff5fddee,0x30a0b3acec89f51db11e1431b4674c988d701c73d148...,80000.000003,False,True
2,2023-09-09T16:36:49.707Z,2023-09-09 16:31:35.000 UTC,0xecf5e00c15d4f76c1ab1e522327c530d104bdcd7,0x8800f541015615d6d987eb1b9689d5e18ad98e1b,0xb4899ac5a9239aa3c56324b187b35dd70ac2856727ae...,80000.000000,False,True
3,2023-09-11T09:21:09.459Z,2023-09-11 09:09:10.000 UTC,0xc1ac969049bd8e9ef91cd9751b59f57c52e766e7,0x7856acbfe8f2a4f179548d8c7663e844ef5c47ca,0x08f062954656c2c23a2ab07b9c4e294f71c64516910a...,75000.590000,False,True
4,2023-09-11T10:37:58.927Z,2023-09-11 10:00:40.000 UTC,0x6c95eebeaa1c9637f9af6c55bf9ff0e414fee705,0x54a78edfedcfa220dd3ea9b407bc372e65cf02e0,0x3dfc0ea1685a48fc945b39aaecdedfad2d67da4156a0...,75000.000000,False,True


In [122]:
# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

while nr_index < len(nr_waitlist_01) and stake_index < len(stake_waitlist_01):
    nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
    stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]

    final_chunks.extend([nr_chunk, stake_chunk])
    
    nr_index += nr_chunk_size
    stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

display(final_waitlist.head())

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-11T14:46:19.272Z,2023-09-04 00:47:25.000 UTC,0xf7adb6cd145c846c3805dc2cd57f1ca6a1800803,0xe540efb03334e43a9bf33c791819c279f8342ecb,0x53f5160b9f9d5e970141f841833862bebdfdb4a916c7...,10000.000000,True,True
1,2023-09-09T14:13:15.202Z,2023-09-04 14:55:25.000 UTC,0x26323877aab52f90c5da19b53320c99627ab2bfc,0x8629dbfd74e0b6016215c064093e3c8259a3b260,0x64750473ebf6c7d3b9d53483f4e89a32a43e3ab8a166...,11425.346894,True,True
2,2023-09-12T20:19:18.691Z,2023-09-04 17:27:20.000 UTC,0xFB2165f4AbEEd496B4EDd3234C4CC09f1557226e,0x3093a8e1621029608e8759f15b74fb18044723ab,0xc8e4ce1c4c9f147a6321d8bd598c355ba5a2095188ca...,26650.897940,True,True
3,2023-09-11T10:29:27.173Z,2023-09-05 17:34:20.000 UTC,0x9bd3c1c34a5599a3e0789453ae1f5f50fbc78c27,0x17b594192539e703febcb218357157d1b6378c7f,0x3ccbc34d480097cd8c2de2ee65498070589f37c6272f...,17069.339900,True,True
4,2023-09-11T13:41:24.101Z,2023-09-06 08:33:40.000 UTC,0xc65525f2Af82eB3D0a5ACBB54b3f7771860f61e0,0x9bf3b5e7f632b60c98836e059558895d3db08556,0x8bcfe5ffee530413ee7dad0f5b53f39d655d6880b02a...,12548.660373,True,True


### Save final waitlist 

In [123]:
final_waitlist.to_excel('final_waitlist.xlsx', index=False)